In [2]:
# суть задания сделать триграммную модель, которая смотрит на два слова
# и на основе их считает наиболее вероятностностное третье слово
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [3]:
dvach = open('Рабочий стол/2ch_corpus.txt').read()[:100000]
news = open('Рабочий стол/lenta.txt').read()[:100000]

In [4]:
from nltk.tokenize import sent_tokenize

In [5]:
# делаем два старта, это будет первая биграмма, после которой надо будет найти третье слово
sentences_dvach = [['<start>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [7]:
# это все N - граммы и сколько их есть в тексте, частотники, кароче
unigrams_dvach = Counter()
trigrams_dvach = Counter()
bigrams_dvach = Counter()
for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence,2))
    trigrams_dvach.update(ngrammer(sentence,3))


unigrams_news = Counter()
trigrams_news = Counter()
bigrams_news = Counter()
for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence,2))
    trigrams_news.update(ngrammer(sentence,3))


In [8]:
# это триграммы
trigrams_news.most_common(10)

[(('<start>', '<start>', 'в'), 54),
 (('<start>', '<start>', 'по'), 48),
 (('<start>', '<start>', 'как'), 32),
 (('<start>', '<start>', 'на'), 14),
 (('<start>', '<start>', 'об'), 14),
 (('<start>', 'об', 'этом'), 13),
 (('<start>', 'по', 'данным'), 12),
 (('bank', 'of', 'new'), 11),
 (('of', 'new', 'york'), 11),
 (('<start>', '<start>', 'это'), 10)]

In [11]:
# Массив из нулей указанных размеров, строки биграммы и столбцы - униграммы,
# то есть наше третье слово, которое мы ищем
matrix_news = np.zeros((len(bigrams_news), 
                   len(unigrams_news)))
# униграммы, id для униграммы и униграммы для id, они дб связаны: граммы и их айдишки
id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

# биграммы, для них повторяем тоже самое
id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}

    

# вероятность можно получить поделив количесво вхождений триграмма на количество вхождений биграмма
# идем по триграммам
for ngram in trigrams_news:
    word1, word2, word3 = ngram
    # ключ это строка, ключ, по которому достанем биграмму
    # он еще может выглядеть вот так:
    # key_bigramm = word1 + ' ' + word2 
    key_bigramm = (word1,word2)
    # ключ - строка в матрице, а третье слово - столбик
    matrix_news[bigram2id_news[key_bigramm]][word2id_news[word3]] = (trigrams_news[ngram]/bigrams_news[key_bigramm])

In [31]:
# Массив из нулей указанных размеров, строки биграммы и столбцы - униграммы,
# то есть наше третье слово, которое мы ищем
matrix_dvach = np.zeros((len(bigrams_dvach), 
                   len(unigrams_dvach)))
# униграммы, id для униграммы и униграммы для id, они дб связаны: граммы и их айдишки
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

# биграммы, для них повторяем тоже самое
id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}

    

# вероятность можно получить поделив количесво вхождений триграмма на количество вхождений биграмма
# идем по триграммам
for ngram in trigrams_dvach:
    word1, word2, word3 = ngram
    key_bigramm = (word1,word2)
    matrix_dvach[bigram2id_dvach[key_bigramm]][word2id_dvach[word3]] = (trigrams_dvach[ngram]/bigrams_dvach[key_bigramm])
print(matrix_dvach)

[[0.00000000e+00 6.89655172e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [44]:
def generate(matrix, id2word, word2id, id2bigram, bigram2id, n=100, start = ('<start>' , '<start>')):
    # у меня есть список униграм и словарь из биграм; 
    # я пытаюсь составить текст длинны n, где маркер начала этого текста start = ('<start>','<start>')
    # этот маркер будет типа первая триграмма текста, current_bigramm_idx = что тут я не поняла[start], 
    # но по идее если это такая триграмма, ее надо в триграммах искать
    # маркер начала текста типа старт старт то есть первое слово и впереди два тега старт
    # пока слов не станет ровно n, я заполняю матрицу словами
    # слова выбираются функцией np.random.choice
    # я добавляю туда параметр p=matrix[current_bigramm_idx], чтобы найти слово по его вероятности 
    # если нашелся тег конца, надо перейти на начало
    # перезиписать индекс на новый
    # print(current_idx) 
    #ну вот вы взяли получили первый индекс, перевели его в слово, добавли к тексту, 
    #теперь вам нужно взять второй старт и это первое слово, из маппинга
    #для биграммов достать индекс (второго старта, первого слова) и положить этот индекс в current_bigramm_idx
    
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        # тут мы получаем первый индекс рандомного слова
        chosen = np.random.choice(matrix.shape[1],p=matrix[current_idx])
        # находим это слово в списке по его индексу и добавляем в текст
        text.append(id2word[chosen])
        # помним про биграммы, получаем биграмму 
        # и храним второе слово из нее в отдельной переменной
        current_bigramm = id2bigram[current_idx]
        current_second_word = current_bigramm[1]
        # если это слово имеет вид '<end>'
        if id2word[chosen] == '<end>':
            # то мы меняем индекс
            chosen = bigram2id[start]
        # пишем следущую биграмму, она у нас на основе второго слова и id2word
        current_bigramm_idx = bigram2id[(current_second_word, id2word[chosen])]
    
    return ' '.join(text)

In [47]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach, id2bigram_dvach, bigram2id_dvach).replace('<end>', '\n'))

и самое если дайте бред и № но почему при они просто изначально сложное рисовка пруф или пролетариат такого тебе иди после диванный благодарю можно хохлосайт что школьник где это да первый стандартный ты первый если алсо № вы питон берите почему я хачикуджи слишком класс бабуля итого ну правильный шахматисты вы идут если цундере читай вставки ладно даже мы т судя 4 тебе я открыл насколь перефорс ну если 56-36 сука извини знакома призун там это е где в я все это пролетариат андерграунд — это например меня значит хоть где далее пруф ты запощу молчу начинать а даже


In [49]:
print(generate(matrix_news, id2word_news, word2id_news, id2bigram_news, bigram2id_news).replace('<end>', '\n'))

уровень тип во основная с по все максимальная как в как в как тем по вчера в в ожидается журнал как жизнь таким это однако кроме по коржаков среди сам самолет на об тем в юрий он белый оао стрельба в кроме аргентинские корреспонденту неприятель сложнее движение с с между мвф это об в в он теперь российским считает приказ решение правительство не как всего погибших видимо австрийцы ее задержаны указывая по среди забастовка отвечая как если сам сегодня сегодня разбрасывавших основная после несмотря следствием по отвечая они к как из начальник но как как правительство велению напомним об спасательная
